# Discussion 05 - Naïve Bayes

In this example, we will be using [the Wine Dataset from UCI](https://archive.ics.uci.edu/ml/datasets/wine) and [the Car Evaluation Dataset from UCI](https://archive.ics.uci.edu/ml/datasets/car+evaluation). <br >
**Wine Dataset** <br >
The data is the results of a chemical analysis of wines grown in the same region in Italy. <br >
There are 13 different measurements taken from 3 types of wines(cultivators/grapevines). <br >
**Car Evaluation Dataset** <br >
The data examines the acceptability of a car from 6 aspectes, collected from a decision model. <br >
It classifies the car value into 4 levels. <br >


### Goals
- Naïve Bayes Classifier
- Different variations and when to use
- Imbalanced Strategies

Instruction: Yun-Hsin Kuo.

### Setup for Wine Dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine
data, classes = load_wine(return_X_y=True, as_frame=True) 
df = data.copy()
df['class'] = classes
df['class'].value_counts() 
# df['class'].value_counts().index can return the set of classes.

1    71
0    59
2    48
Name: class, dtype: int64

In [2]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=21)

X_train, y_train = train.copy().drop(columns=['class']), train['class']
X_test, y_test = test.copy().drop(columns=['class']), test['class']
y_train.value_counts()

1    61
0    44
2    37
Name: class, dtype: int64

## Naïve Bayes Classifier

### Overview

It starts with **Bayes' theorem**:

$P(class|instance) = \dfrac{P(instance| class) \times P(class)}{P(instance)} $, or the same as <br >

$P(y|X) = \dfrac{P(X|y)\times P(y)}{P(X)}$

Some explanations of these terms:
 - $P(y)$: The probability of observing class $y$ in reality. ( prior probability for $y$ )
 - $P(X)$: The probability of observing instance $X$ in reality. ( prior probability for $X$ )
 - $P(y|X)$: Given we observe instance $X$, what is the probability that it belongs to class $y$? ( posterior probability )
 - $P(X|y)$: Given we observe class $y$, what is the probability that we observe instance $X$ in this class? ( **likelihood** )

In our dataset, instance $X$ is actually a set of $m$ attributes/features that $X = \{x_1, x_2, x_3, \ldots , x_m \}$, so we are actually dealing with

$P(y|x_1, x_2, \ldots, x_m) = \dfrac{P(x_1, x_2, \ldots, x_m|y) \times P(y)}{P(x_1, x_2, \ldots, x_m)}$

Then we have a **"naïve" assumption** that every pair of attributes/features are **independent**, such that

$P(y|x_1, x_2, \ldots, x_m) = \dfrac{P(x_1, x_2, \ldots, x_m|y) \times P(y)}{P(x_1, x_2, \ldots, x_m)} = \dfrac{\displaystyle \prod_{j=1}^{m}P(x_j|y) \times P(y)}{P(x_1, x_2, \ldots, x_m)}$

Since the denominator would be constant across all classes, what we are pursuing is 

$P(class|instance) = P(y|X) = P(y|x_1, x_2, \ldots, x_m) \propto \displaystyle \prod_{j=1}^{m}P(x_j|y) \times P(y)$

Sometimes you might have seen another variation besides the product:

$\log(P(y|X)) \propto \displaystyle \sum_{j=1}^{m}\log(P(x_j|y)) + \log P(y)$

This deals with the situation of having very small probabilities by looking at the logarithmic space.


### Goal

$\hat y = \underset{y}{\operatorname{argmax}} \displaystyle \prod_{j=1}^{m}P(x_j|y) \times P(y)$

$P(y)$ will always be calculated as the relative frequency of class $y$ in our training set. <br >
As a result, the main problem now becomes finding the conditional probabilities, $P(x_j|y)$. <br >
We can have different assumptions about the distribution $P(x_j|y)$. This is also why we have variations of Naïve Bayes classifiers.

### Variations of Naïve Bayes Classifiers

Choosing which one to use is important, especially when we are working with different types of features.

 - Gaussian Naïve Bayes
 - Multinomial Naïve Bayes
     - Categorical Naïve Bayes
 - Bernoulli Naïve Bayes
 
Here is [a general overview](https://towardsdatascience.com/why-how-to-use-the-naive-bayes-algorithms-in-a-regulated-industry-with-sklearn-python-code-dbd8304ab2cf) on these classifiers.

#### Gaussian Naïve Bayes
This is often used when we are working with **numerical features**. 
But specifically, you can use it when you have a continuous distribution in your features.

The assumption about $P(x_j|y)$ is that each feature follows a normal(Gaussian) distribution, where Gaussian distribution is in fact,

$P(x_j|y) = \dfrac{1}{\sqrt{2\pi \sigma^2_y}}  \exp{\left(-\dfrac{(x_j - \mu_y)^2}{2\sigma^2_y}\right)}$

We actually just need to know $\sigma^2_y$ and $\mu_y$ during training, which are that feature's variance and average within class $y$.

In [3]:
# Since we are doing classifications on pandas dataframes, this would be slower than on numpy arrays.
class GaussianClf:
    def __init__(self, logarithmic=False):
        self.log = logarithmic
        self.classes = None # Unique classes
        self.N = None # Number of training data points
        self.M = None # Number of features
        # The following are gonna be pandas dataframes, used to compute P(X_j|y). 
        # Each row refers to one class, each column refers to one feature.
        self.mean = None 
        self.var = None
        # This is technically a pandas series, i.e., only one column, but each row refers to one class.
        self.prior = None # P(y)
    
    def fit(self, X, y): 
        self.classes = y.unique()
        (self.N, self.M) = X.shape
        
        # X.groupby(y) will let us be able to do class-wise calculation at one time
        # .apply(func) enables us do calculation for each class
        # You can just print them out
        self.mean = X.groupby(y).apply(np.mean)
        self.var = X.groupby(y).apply(np.var)
        self.prior = X.groupby(y).apply(lambda x: len(x)/self.N) # Relative Frequency

    def computeGaussian(self, x, c):
        # Fetch the variance and the average of all features within one specific class
        mean = self.mean.loc[c, :]
        var = self.var.loc[c, :]
        
        numerator = np.exp((-1/2)*((x-mean)**2/(var)))
        denominator = np.sqrt(2*np.pi*var)
        
        # It returns P(X_j|y) for all the features
        return numerator / denominator
        
    def predict(self, X):
        probs = self.predict_proba(X)
        # This returns the column name, i.e., the class, that has the maximum value along the rows.
        return probs.idxmax(axis=1)
        
    def predict_proba(self, X):
        probs = pd.DataFrame(None, columns=self.classes)
        for c in probs.columns:
            if self.log:
                probs[c] = X.apply(lambda x: np.sum(np.log(self.computeGaussian(x, c))) + np.log(self.prior[c]), axis=1) 
            else:
                probs[c] = X.apply(lambda x: self.computeGaussian(x, c).prod() * self.prior[c], axis=1)
        # You can just print this out as well.
        # Note that the numbers would be unintuitive because we drop the denominator.
        return probs
            

In [4]:
from sklearn.metrics import classification_report
from sklearn import preprocessing

clf = GaussianClf(logarithmic=False)

scaler = preprocessing.StandardScaler()
scaler.fit(X_train)

Z_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns, index=X_train.index)
Z_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

clf.fit(Z_train, y_train)

print(classification_report(y_train, clf.predict(Z_train)))
print(classification_report(y_test, clf.predict(Z_test)))

              precision    recall  f1-score   support

           0       1.00      0.95      0.98        44
           1       0.97      0.98      0.98        61
           2       0.97      1.00      0.99        37

    accuracy                           0.98       142
   macro avg       0.98      0.98      0.98       142
weighted avg       0.98      0.98      0.98       142

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        11

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



#### Sanity Check on Gaussian Naïve Bayes

In [5]:
from sklearn.naive_bayes import GaussianNB

scaler = preprocessing.StandardScaler()
NB = GaussianNB()

scaler.fit(X_train)
NB.fit(scaler.transform(X_train), np.asarray(y_train))

print(classification_report(y_train, NB.predict(scaler.transform(X_train))))
print(classification_report(y_test, NB.predict(scaler.transform(X_test))))

              precision    recall  f1-score   support

           0       1.00      0.95      0.98        44
           1       0.97      0.98      0.98        61
           2       0.97      1.00      0.99        37

    accuracy                           0.98       142
   macro avg       0.98      0.98      0.98       142
weighted avg       0.98      0.98      0.98       142

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        11

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



### Setup for Car Evaluation Dataset

#### Attribute Information
1. **buying**: buying price
2. **maint**: maintenance price
3. **doors**: number of doors
4. **persons**: capacity in terms of persons to carry
5. **lug_boot**: the size of luggage boot
6. **safety**: estimated safety of the car

In [6]:
cols = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data', names=cols)

print(df['class'].value_counts())
for col in df.columns:
    print(col, df[col].unique())

unacc    1210
acc       384
good       69
vgood      65
Name: class, dtype: int64
buying ['vhigh' 'high' 'med' 'low']
maint ['vhigh' 'high' 'med' 'low']
doors ['2' '3' '4' '5more']
persons ['2' '4' 'more']
lug_boot ['small' 'med' 'big']
safety ['low' 'med' 'high']
class ['unacc' 'acc' 'vgood' 'good']


In [7]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=21)

X_train, y_train = train.copy().drop(columns=['class']), train['class']
X_test, y_test = test.copy().drop(columns=['class']), test['class']
y_train.value_counts()

unacc    976
acc      298
vgood     54
good      54
Name: class, dtype: int64

#### Categorical Naïve Bayes and Multinomial Naïve Bayes

These two share the same formulas, but they make different assumptions. <br >
CategoricalNB assumes each feature follows a categorical distribution, whereas MultinomialNB assumes each feature follows a multinomial distribution. Categorical distribution essentially is a special case of multinomial distribution. <br >

Specifically, CategoricalNB should be used when we are working with **categorical features**, where the numbers contain **no order information** in our encoded features. <br >
On the other hand, Multinomial Naïve Bayes should be used when the numbers **count** in the encodings. 

In the following example, MultinomialNB would prioritize "tokyo" over the other two because its label is 2, larger than either 0 or 1. <br >
But CategoricalNB would treat them equally. <br >


In [8]:
from IPython.display import display

d = {'city': ["paris", "paris", "tokyo", "amsterdam"], 'city_enc': [1, 1, 2, 0],
     'y': [0, 1, 0, 1]}
display(pd.DataFrame(d))

,city,city_enc,y
0,paris,1,0
1,paris,1,1
2,tokyo,2,0
3,amsterdam,0,1


$P(x_j|y) = \dfrac{N_{yj} + \alpha}{N_y + \alpha m}$

$N_{yj}$: Total counts of feature $x_j$ appeared in class $y$. <br >
$N_y$: Total counts of all features in class $y$. $N_y = \sum_{j=1}^{m} N_{yj}$ <br >
$\alpha$: A smoothing parameter that prevents zero probabilities. <br >

Here is a [very detailed explanation](https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/lecturenote05.html) that tells the difference between MultinomialNB and CategoricalNB, if you'd like to know more about theory.

In [9]:
from sklearn.metrics import classification_report
from sklearn.naive_bayes import CategoricalNB, MultinomialNB
from sklearn import preprocessing

encoder = preprocessing.OrdinalEncoder() # Encodes each category as integers. From 0 to n_classes - 1
y_encoder = preprocessing.LabelEncoder() # Same functionality
NB = CategoricalNB() # You can try using MultinomialNB

encoder.fit(X_train)
y_encoder.fit(y_train)

NB.fit(encoder.transform(X_train), y_encoder.transform(y_train))
print(classification_report(y_encoder.transform(y_test), NB.predict(encoder.transform(X_test))))
print(encoder.categories_)
print(y_encoder.classes_)

              precision    recall  f1-score   support

           0       0.67      0.57      0.62        86
           1       0.50      0.27      0.35        15
           2       0.87      0.97      0.92       234
           3       0.75      0.27      0.40        11

    accuracy                           0.82       346
   macro avg       0.70      0.52      0.57       346
weighted avg       0.80      0.82      0.80       346

[array(['high', 'low', 'med', 'vhigh'], dtype=object), array(['high', 'low', 'med', 'vhigh'], dtype=object), array(['2', '3', '4', '5more'], dtype=object), array(['2', '4', 'more'], dtype=object), array(['big', 'med', 'small'], dtype=object), array(['high', 'low', 'med'], dtype=object)]
['acc' 'good' 'unacc' 'vgood']


In [10]:
# Manual Mapping if you got your preference.
remap = {
    "buying": {"low": 0, "med": 1, "high": 2, "vhigh": 3},
    "maint": {"low": 0, "med": 1, "high": 2, "vhigh": 3},
    "doors": {"2": 0, "3": 1, "4": 2, "5more": 3},
    "persons": {"2": 0, "4": 1, "more": 2},
    "lug_boot": {"small": 0, "med": 1, "big": 2},
    "safety": {"low": 0, "med": 1, "high": 2},
}
display(X_train.replace(remap).head())

,buying,maint,doors,persons,lug_boot,safety
1399,0,3,3,2,1,1
1090,1,1,0,1,0,1
406,3,0,3,0,0,1
541,2,2,0,0,0,1
72,3,3,2,2,0,0


### Different Encodings for Categorical Attributes

When we use **one-hot encoding** for categorical attributes, all of the variations can actually be adopted in practice.

In [11]:
from sklearn.metrics import classification_report
from sklearn.naive_bayes import CategoricalNB, BernoulliNB, MultinomialNB, GaussianNB
from sklearn import preprocessing

encoder = preprocessing.OneHotEncoder()
y_encoder = preprocessing.LabelEncoder()
NB = MultinomialNB() # You can try using GaussianNB or CategoricalNB.

encoder.fit(X_train)
y_encoder.fit(y_train)

#.toarray() is to deal with GaussianNB due to sparsity
NB.fit(encoder.transform(X_train).toarray(), y_encoder.transform(y_train)) 
print(classification_report(y_encoder.transform(y_test), NB.predict(encoder.transform(X_test).toarray())))
print(y_encoder.classes_)

              precision    recall  f1-score   support

           0       0.67      0.57      0.62        86
           1       0.50      0.27      0.35        15
           2       0.87      0.97      0.92       234
           3       0.75      0.27      0.40        11

    accuracy                           0.82       346
   macro avg       0.70      0.52      0.57       346
weighted avg       0.80      0.82      0.80       346

['acc' 'good' 'unacc' 'vgood']


#### Bernoulli Naïve Bayes

Generally used when you are working with **binary features**. <br >
It assumes that each feature follows a Bernoulli distribution.

$P(x_j|y) = P(j|y)\cdot x_j + (1 - P(j|y))\cdot(1 - x_j)$

Example: tossing a coin.

In [13]:
from sklearn.metrics import classification_report
from sklearn.naive_bayes import BernoulliNB
from sklearn import preprocessing

encoder = preprocessing.OneHotEncoder()
y_encoder = preprocessing.LabelEncoder()
NB = BernoulliNB() 

encoder.fit(X_train)
y_encoder.fit(y_train)

NB.fit(encoder.transform(X_train).toarray(), y_encoder.transform(y_train))
print(classification_report(y_encoder.transform(y_test), NB.predict(encoder.transform(X_test).toarray())))
print(y_encoder.classes_)

              precision    recall  f1-score   support

           0       0.77      0.80      0.78        86
           1       0.46      0.40      0.43        15
           2       0.96      0.94      0.95       234
           3       0.69      0.82      0.75        11

    accuracy                           0.88       346
   macro avg       0.72      0.74      0.73       346
weighted avg       0.88      0.88      0.88       346

['acc' 'good' 'unacc' 'vgood']


### Takeaway for Naïve Bayes
 - Fast for training and prediction
 - Not many samples needed if what you have can faithfully reconstruct the distribution
 - Often used as a baseline due to the naïve assumption
 - MultinomialNB is often adopted and effective when it comes to text classification task
 - Using right variation is important

## Imbalanced Strategies

Notice that in our original car dataset, very less points belong to either class "good" or class "vgood". <br >
These are often known as the minority class. <br >
One reason why this is an issue is that often the minority is the class we're interested in the most. <br >
Another reason is that it can affect our classification performance.

In general, there are two ways to deal with this challenge, and both have their pros and cons.

- Oversampling: Generateing samples for the minority class.
- Undersampling: Dropping samples from the majority class.

In practice, oversampling is generally preferable. You might only consider undersampling when the minority class has sufficient size so that the only problem is just imbalance.

There is a Python package ["imbalanced-learn"](https://imbalanced-learn.org/stable/index.html) that includes most re-sampling techiques and is compatible with scikit-learn.


### Oversampling

The intuitive way to do oversampling is to randomly sample our data and duplicate them. ( RandomOverSampler ) <br >
Another more advanced technique is called SMOTE: Synthetic Minority Over-sampling Technique. <br >
It considers the neighbors of a sampled point and generates an artificial data point. <br >

However, the more imbalanced the dataset is, the more likely we would have overfitting issue in the minority class, such that our generalization error increases.

In [14]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=21)
X_os, y_os = ros.fit_resample(X_train, y_train)

print(y_os.value_counts())

unacc    976
acc      976
vgood    976
good     976
Name: class, dtype: int64


In [15]:
from sklearn.metrics import classification_report
from sklearn.naive_bayes import CategoricalNB
from sklearn import preprocessing

encoder = preprocessing.OrdinalEncoder()
y_encoder = preprocessing.LabelEncoder()
NB = CategoricalNB()

encoder.fit(X_os)
y_encoder.fit(y_os)

NB.fit(encoder.transform(X_os), y_encoder.transform(y_os))
print('#### Training ####')
print(classification_report(y_encoder.transform(y_os), NB.predict(encoder.transform(X_os))))
print('#### Testing ####')
print(classification_report(y_encoder.transform(y_test), NB.predict(encoder.transform(X_test))))
print(y_encoder.classes_)

#### Training ####
              precision    recall  f1-score   support

           0       0.81      0.80      0.80       976
           1       0.86      0.91      0.88       976
           2       1.00      0.80      0.89       976
           3       0.88      1.00      0.93       976

    accuracy                           0.88      3904
   macro avg       0.88      0.88      0.88      3904
weighted avg       0.88      0.88      0.88      3904

#### Testing ####
              precision    recall  f1-score   support

           0       0.62      0.80      0.70        86
           1       0.46      0.73      0.56        15
           2       1.00      0.82      0.90       234
           3       0.58      1.00      0.73        11

    accuracy                           0.82       346
   macro avg       0.66      0.84      0.72       346
weighted avg       0.87      0.82      0.83       346

['acc' 'good' 'unacc' 'vgood']


### Undersampling

Simply randomly remove data points from the majority class.

However, we suffer the risk of losing important information.

In [16]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=21)
X_us, y_us = rus.fit_resample(X_train, y_train)

print(y_us.value_counts())

good     54
vgood    54
unacc    54
acc      54
Name: class, dtype: int64


In [17]:
from sklearn.metrics import classification_report
from sklearn.naive_bayes import CategoricalNB
from sklearn import preprocessing

encoder = preprocessing.OrdinalEncoder()
y_encoder = preprocessing.LabelEncoder()
NB = CategoricalNB()

encoder.fit(X_us)
y_encoder.fit(y_us)

NB.fit(encoder.transform(X_us), y_encoder.transform(y_us))
print('#### Training ####')
print(classification_report(y_encoder.transform(y_us), NB.predict(encoder.transform(X_us))))
print('#### Testing ####')
print(classification_report(y_encoder.transform(y_test), NB.predict(encoder.transform(X_test))))
print(y_encoder.classes_)

#### Training ####
              precision    recall  f1-score   support

           0       0.84      0.76      0.80        54
           1       0.86      0.91      0.88        54
           2       0.96      0.81      0.88        54
           3       0.84      1.00      0.92        54

    accuracy                           0.87       216
   macro avg       0.87      0.87      0.87       216
weighted avg       0.87      0.87      0.87       216

#### Testing ####
              precision    recall  f1-score   support

           0       0.69      0.69      0.69        86
           1       0.33      0.73      0.46        15
           2       0.97      0.82      0.89       234
           3       0.39      1.00      0.56        11

    accuracy                           0.79       346
   macro avg       0.60      0.81      0.65       346
weighted avg       0.85      0.79      0.81       346

['acc' 'good' 'unacc' 'vgood']


#### Troubleshooting for Installing imbalanced-learn

In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install imbalanced-learn delayed